In [7]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns 

In [8]:
banki=pd.read_csv('Dane_bank_nowe.csv', sep=',')
banki = banki.drop(["Unnamed: 0", 'RowNumber'], axis=1)

In [15]:
banki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


In [10]:
banki_cp=banki.copy(deep=True)

In [19]:
def div_2(x):
    if x > 640:
        return 0
    else:
        return 1

banki['CreditScore_2']=banki.apply(lambda x: div_2(x['CreditScore']),axis=1 )

In [20]:
banki.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,CreditScore_2
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,886607.90,1,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,916554.56,0,1
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,849781.25,1,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,1367384.50,0,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,1504164.20,0,0


In [ ]:
def div_5(x):
    if x < 580:
        return 0
    elif x>580 and x<640:
        return 1
    elif x>640 and x<720:
        return 2
    elif x>720 and x<780:
        return 3
    else:
        return 4

banki['CreditScore_4']=banki.apply(lambda x: div_2(x['CreditScore']),axis=1 )
banki.head()